<a href="https://colab.research.google.com/github/Amirnaderiy/CNN_ViT/blob/main/ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 9.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import keras
from keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, StratifiedKFold


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
data_directory = '/content/drive/My Drive/stylegan2/New folder (3)/'


class_A_images = []
class_B_images = []
labels_A = []
labels_B = []

for class_name in os.listdir(data_directory):
    class_path = os.path.join(data_directory, class_name)
    if class_name == 'A':
        for image_filename in os.listdir(class_path):
            image_path = os.path.join(class_path, image_filename)
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            class_A_images.append(image)
            labels_A.append(0)  # Class A label
    elif class_name == 'B':
        for image_filename in os.listdir(class_path):
            image_path = os.path.join(class_path, image_filename)
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            class_B_images.append(image)
            labels_B.append(1)  # Class B label


# Convert to numpy arrays
data_A = np.array(class_A_images)
data_B = np.array(class_B_images)
labels_A = np.array(labels_A)
labels_B = np.array(labels_B)

# Combine data from both classes
data = np.concatenate((data_A, data_B), axis=0)
labels = np.concatenate((labels_A, labels_B), axis=0)

In [6]:
test_directory = '/content/drive/My Drive/stylegan2/Test/'


test_A_images = []
test_B_images = []
testlabels_A = []
testlabels_B = []

for class_name in os.listdir(test_directory):
    class_path = os.path.join(test_directory, class_name)
    if class_name == 'A':
        for image_filename in os.listdir(class_path):
            image_path = os.path.join(class_path, image_filename)
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            test_A_images.append(image)
            testlabels_A.append(0)  # Class A label
    elif class_name == 'B':
        for image_filename in os.listdir(class_path):
            image_path = os.path.join(class_path, image_filename)
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            test_B_images.append(image)
            testlabels_B.append(1)  # Class B label


# Convert to numpy arrays
testdata_A = np.array(test_A_images)
testdata_B = np.array(test_B_images)
testlabels_A = np.array(testlabels_A)
testlabels_B = np.array(testlabels_B)

# Combine data from both classes
test_data = np.concatenate((testdata_A, testdata_B), axis=0)
test_labels = np.concatenate((testlabels_A, testlabels_B), axis=0)

In [7]:
valid_directory = '/content/drive/My Drive/stylegan2/valid/'


valid_A_images = []
valid_B_images = []
validlabels_A = []
validlabels_B = []

for class_name in os.listdir(valid_directory):
    class_path = os.path.join(valid_directory, class_name)
    if class_name == 'A':
        for image_filename in os.listdir(class_path):
            image_path = os.path.join(class_path, image_filename)
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            valid_A_images.append(image)
            validlabels_A.append(0)  # Class A label
    elif class_name == 'B':
        for image_filename in os.listdir(class_path):
            image_path = os.path.join(class_path, image_filename)
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            valid_B_images.append(image)
            validlabels_B.append(1)  # Class B label


# Convert to numpy arrays
validdata_A = np.array(valid_A_images)
validdata_B = np.array(valid_B_images)
validlabels_A = np.array(validlabels_A)
validlabels_B = np.array(validlabels_B)

# Combine data from both classes
valid_data = np.concatenate((validdata_A, validdata_B), axis=0)
valid_labels = np.concatenate((validlabels_A, validlabels_B), axis=0)

In [16]:





num_classes = 2
input_shape = (120, 120, 3)


# Configure the hyperparameters
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 120
image_size = 120 # We'll resize input images to this size
patch_size = 6 # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
] # Size of the transformer layers
transformer_layers = 4
mlp_head_units = [2048, 1024] # Size of the dense layers of the final classifier

# Use data augmentation
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),  # Match the input size to image_size
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)


# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(data)

# Implement multilayer perceptron (MLP)
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

# Implement patch creation as a layer
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

# Implement the patch encoding layer
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

# Build the ViT model
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

# Compile the model
model = create_vit_classifier()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    ),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="accuracy")
    ],
)

# Train the model
data_resized = tf.image.resize(data, (120, 120))
history = model.fit(
    x=data_resized,
    y=labels,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_data=(valid_data, valid_labels),
)

# Evaluate the model
model.evaluate(test_data, test_labels)

Epoch 1/120
15/15 [==============================] - 25s 671ms/step - loss: 16.5989 - accuracy: 0.6511 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 2/120
15/15 [==============================] - 9s 632ms/step - loss: 6.2475 - accuracy: 0.6267 - val_loss: 8.7505 - val_accuracy: 0.4000
Epoch 3/120
15/15 [==============================] - 3s 210ms/step - loss: 2.7210 - accuracy: 0.7133 - val_loss: 0.4494 - val_accuracy: 0.8200
Epoch 4/120
15/15 [==============================] - 10s 709ms/step - loss: 1.4339 - accuracy: 0.7667 - val_loss: 0.4709 - val_accuracy: 0.8400
Epoch 5/120
15/15 [==============================] - 4s 272ms/step - loss: 1.3407 - accuracy: 0.7778 - val_loss: 0.0398 - val_accuracy: 0.9600
Epoch 6/120
15/15 [==============================] - 9s 611ms/step - loss: 1.1458 - accuracy: 0.7867 - val_loss: 0.0630 - val_accuracy: 0.9600
Epoch 7/120
15/15 [==============================] - 12s 820ms/step - loss: 0.8867 - accuracy: 0.8489 - val_loss: 0.5688 - val_accuracy: 0.

[6.346757411956787, 0.4399999976158142]

In [18]:
model.evaluate(test_data, test_labels)

2/2 [==============================] - 1s 362ms/step - loss: 6.3468 - accuracy: 0.4400


[6.346757411956787, 0.4399999976158142]